In [1]:
import numpy as np

In [45]:
def fit(X,y):
    dictionary = {}
    classes = set(y)
    features = X.shape[1]
    dictionary['total_classes'] = len(y)
    for i in classes:
        current_class_rows = (y == i)
        X_current = X[current_class_rows]
        y_current = y[current_class_rows]
        class_occurence = len(y_current)
        dictionary[i] = {}
        dictionary[i]['class_occurence'] = class_occurence
        for j in range(1,features + 1):
            dictionary[i][j] = {}
            t = set(X[:,j-1])
            for k in t:
                dictionary[i][j][k] = (X_current[:,j-1] == k).sum()
                
    return dictionary

In [3]:
y = np.array([1,2,3,4,4])
len(y[y==4])

2

In [48]:
def prediction(dictionary,x,y):
    output = np.log(dictionary[y]['class_occurence']) - np.log(dictionary['total_classes'])
    num_features = len(dictionary[y].keys()) - 1
    for i in range(1,num_features+1):
        c = x[i-1] 
        current_value = dictionary[y][i][c] + 1
        count_current_class = dictionary[y]['class_occurence'] + len(dictionary[y].keys())
        probability = np.log(current_value) - np.log(count_current_class)
        output = output + probability
    return output

    
    

In [47]:
def bestPredSinglePoint(dictionary,x):
    classes = dictionary.keys()
    best = -1000
    bestClass = -12
    first_run = True
    for i in classes:
        if i == 'total_classes':
            continue
        pred = prediction(dictionary,x,i)
        if first_run or pred > best :
            best = pred
            bestClass = i
        first_run = False
    return bestClass
    
    
    

In [50]:
def pred(dictionary,X_test):
    y_pred = []
    for i in X_test:
        p = bestPredSinglePoint(dictionary,i)
        y_pred.append(p)
    return y_pred   

In [7]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0,len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column


In [8]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [9]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [10]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [51]:
dictionary = fit(X_train,Y_train)
Y_pred = pred(dictionary,X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.94      1.00      0.97        16
          2       1.00      0.89      0.94         9

avg / total       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
